In [ ]:
!wget https://hitachiintel2021.blob.core.windows.net/model/model.xml
!wget https://hitachiintel2021.blob.core.windows.net/model/model.bin

In [ ]:
!ls

In [ ]:
!pbsnodes | grep "properties =" | sort | uniq -c

In [ ]:
!pwd
!cp -r $INTEL_OPENVINO_DIR/deployment_tools/tools/benchmark_tool .
!ls -l benchmark_tool

## 計算ノードで実行するスクリプトを作成 

In [ ]:
%%writefile job.sh
cd ~/
pip3 install -r benchmark_tool/requirements.txt
python3 benchmark_tool/benchmark_app.py \
        -m model.xml \
        -niter 100 \
        $*
echo completed

## 計算ノードへジョブを投げて、完了までモニタリングするPythonプログラムを作成

In [ ]:
import time
def waitForJobCompletion(jobNumber):
    print('Waiting for job completion...', end='')
    running=True
    while running:
        time.sleep(1)
        running=False
        status_list=!qstat         # Check job status
        for status in status_list:
            if jobNumber in status:    # if job_num is found in the status list, the job is still running
                running = True
        print(status.split()[4], end='')
    print('...Job {} completed'.format(job_num))
    
    # 1つ前のセルでlog_fileにログファイル名を入れている。環境変数にセットしてcatから使えるようにする
    
    os.environ['log_file']=log_file

    !grep '# Resources:'                                  $log_file
    !grep '\[ INFO \] Device info'                   -A 3 $log_file
    !grep '\[Step 11/11\] Dumping statistics report' -A 4 $log_file
    print("---------------------------------------------------------------------------------")

In [ ]:
import os

node_list = ["e3950", "j3355", "i5-7500", "i7-8665ue", "i7-1065g7", "i7-1185g7e", "gold6338n"]
#node_list = ["gold6338n"]

for node_name in node_list:
    os.environ['node_name'] = node_name

    job_id=!qsub -l nodes=1:$node_name job.sh -F "-nireq 1 -nstreams 1 -b 1"

    # generate log file name from job_id
    job_num = job_id[0].split('.')[0]
    log_file='job.sh.o'+job_num
    err_file='job.sh.e'+job_num
    print('job_id={}, log_file={}'.format(job_id, log_file))

    # wait for the job to complete
    waitForJobCompletion(job_num)

## Azure BLOB ストレージへアップロード

In [ ]:
!pip3 install azure-storage-blob

In [ ]:
!pip3 list

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")

    # Quick start code goes here

except Exception as ex:
    print('Exception:')
    print(ex)

# Retrieve the connection string for use with the application. The storage
# connection string is stored in an environment variable on the machine
# running the application called AZURE_STORAGE_CONNECTION_STRING. If the environment variable is
# created after the application is launched in a console or with Visual Studio,
# the shell or application needs to be closed and reloaded to take the
# environment variable into account.
connect_str = "DefaultEndpointsProtocol=https;AccountName=hitachiintel2021;AccountKey=Fr6XqoVLkGObclVigaO5ORZLE3jMpfdEdNsuKke5XApoovO4EyKEhMTm7AhLyp8vrYp4A5QAc52aZ/P/fGiT+A==;BlobEndpoint=https://hitachiintel2021.blob.core.windows.net/;QueueEndpoint=https://hitachiintel2021.queue.core.windows.net/;TableEndpoint=https://hitachiintel2021.table.core.windows.net/;FileEndpoint=https://hitachiintel2021.file.core.windows.net/;"

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Create a unique name for the container
container_name = "model"

# Create a file in the local data directory to upload and download
local_xml_file_name = "model.xml"
local_bin_file_name = "model.bin"

# Create a blob client using the local file name as the name for the blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob="latest/"+local_xml_file_name)

print("\nUploading to Azure Storage as blob:\n\t" + local_xml_file_name)

# Upload the created file
with open(local_xml_file_name, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)


# Create a blob client using the local file name as the name for the blob
blob_client = blob_service_client.get_blob_client(container=container_name, blob="latest/"+local_bin_file_name)

print("\nUploading to Azure Storage as blob:\n\t" + local_bin_file_name)

# Upload the created file
with open(local_bin_file_name, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)
    